# ภารกิจเสริม: การถดถอยเชิงเส้นหลายตัวแปร
ในห้องปฏิบัติการนี้ คุณจะขยายโครงสร้างข้อมูลและขั้นตอนที่พัฒนาขึ้นก่อนหน้านี้เพื่อรองรับหลายคุณลักษณะ ขั้นตอนหลายอย่างได้รับการปรับปรุง ทำให้ห้องปฏิบัติการยาวขึ้น แต่มีการปรับเปลี่ยนเล็กน้อยในขั้นตอนก่อนหน้านี้ ทำให้สามารถตรวจสอบได้อย่างรวดเร็ว


# เนื้อหา
- [&nbsp;&nbsp;1.1 เป้าหมาย](#toc_15456_1.1)
- [&nbsp;&nbsp;1.2 เครื่องมือ](#toc_15456_1.2)
- [&nbsp;&nbsp;1.3  สัญลักษณ์](#toc_15456_1.3)
- [2 ปัญหา](#toc_15456_2)
- [&nbsp;&nbsp;2.1 เมทริกซ์ X ที่มีตัวอย่างของเรา](#toc_15456_2.1)
- [&nbsp;&nbsp;2.2 เวกเตอร์พารามิเตอร์ w, b](#toc_15456_2.2)
- [3 การทำนายแบบจำลองด้วยตัวแปรหลายตัว](#toc_15456_3)
- [&nbsp;&nbsp;3.1 องค์ประกอบการทำนายเดี่ยวตามองค์ประกอบ](#toc_15456_3.1)
- [&nbsp;&nbsp;3.2 การทำนายเดี่ยว เวกเตอร์](#toc_15456_3.2)
- [4 คำนวณ cost ด้วยตัวแปรหลายตัว](#toc_15456_4)
- [5 Gradient Descent การไล่ระดับลงด้วยตัวแปรหลายตัว](#toc_15456_5)
- [&nbsp;&nbsp;5.1 คำนวณการไล่ระดับด้วยตัวแปรหลายตัว](#toc_15456_5.1)
- [&nbsp;&nbsp;5.2 Gradient Descent การไล่ระดับลงด้วยตัวแปรหลายตัว](#toc_15456_5.2)
- [6 ยินดีด้วย](#toc_15456_6)


<a name="toc_15456_1.1"></a>
## 1.1 เป้าหมาย
- ขยายขั้นตอนการถดถอยของเราเพื่อรองรับคุณลักษณะหลายอย่าง
   - ขยายโครงสร้างข้อมูลเพื่อรองรับคุณลักษณะหลายอย่าง
   - เขียนโปรแกรมทำนาย ต้นทุน และการไล่ระดับใหม่เพื่อรองรับคุณลักษณะหลายอย่าง
   - ใช้ np.dot ของ NumPy เพื่อเวคเตอร์การใช้งานเพื่อความเร็วและความเรียบง่าย

<a name="toc_15456_1.2"></a>
## 1.2 เครื่องมือ
ในห้องปฏิบัติการนี้ เราจะใช้:
- NumPy ซึ่งเป็นไลบรารีที่นิยมสำหรับการคำนวณทางวิทยาศาสตร์
- Matplotlib ซึ่งเป็นไลบรารีที่นิยมสำหรับการพล็อตข้อมูล

In [ ]:
# prompt: import requsts and download from this github link : https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle

import requests

url = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle'
response = requests.get(url)

with open('deeplearning.mplstyle', 'wb') as f:
  f.write(response.content)


In [ ]:
import copy, math
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
np.set_printoptions(precision=2)  # reduced display precision on numpy arrays

<a name="toc_15456_1.3"></a>
## 1.3 Notation(สัญลักษณ์)
นี่คือสรุปสัญลักษณ์บางส่วนที่คุณจะพบ ซึ่งได้รับการอัปเดตสำหรับคุณลักษณะหลายรายการ:

|General <img width=70/> <br />  Notation  <img width=70/> | Description<img width=350/>| Python (if applicable) |
|: ------------|: ------------------------------------------------------------||
| $a$ | สเกลาร์ ไม่เป็นตัวหนา                                                      ||
| $\mathbf{a}$ | เวกเตอร์ ตัวหนา                                                 ||
| $\mathbf{A}$ | เมทริกซ์ ตัวหนาอักษรใหญ่                                         ||
| **การถดถอย (Regression)** |         |    |     |
|  $\mathbf{X}$ | เมทริกซ์ตัวอย่างฝึกอบรม                  | `X_train` |   
|  $\mathbf{y}$  | เป้าหมายตัวอย่างฝึกอบรม                | `y_train` 
|  $\mathbf{x}^{(i)}$, $y^{(i)}$ | $i_{th}$ตัวอย่างการฝึกอบรม | `X[i]`, `y[i]`|
| m | จำนวนตัวอย่างการฝึกอบรม | `m`|
| n | จำนวนคุณลักษณะในแต่ละตัวอย่าง | `n`|
|  $\mathbf{w}$  |  พารามิเตอร์: น้ำหนัก (weight),                       | `w`    |
|  $b$           |  พารามิเตอร์: อคติ (bias)                                           | `b`    |     
| $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ | ผลลัพธ์ของการประเมินแบบจำลองที่  $\mathbf{x^{(i)}}$ โดยมีพารามิเตอร์ $\mathbf{w},b$: $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)}+b$  | `f_wb` | 


<a name="toc_15456_2"></a>
# 2 ปัญหา

คุณจะใช้ตัวอย่างการกระตุ้นของการทำนายราคาบ้าน ชุดข้อมูลการฝึกอบรมประกอบด้วยสามตัวอย่างที่มีสี่คุณลักษณะ (ขนาด, ห้องนอน, ชั้น และอายุ) ตามที่แสดงในตารางด้านล่าง โปรดทราบว่า ไม่เหมือนกับห้องปฏิบัติการก่อนหน้านี้ ขนาดเป็นตารางฟุต ไม่ใช่ 1000 ตารางฟุต ซึ่งทำให้เกิดปัญหาที่คุณจะแก้ไขได้ในห้องปฏิบัติการถัดไป!

| Size (sqft) | Number of Bedrooms  | Number of floors | Age of  Home | Price (1000s dollars)  |   
| ----------------| ------------------- |----------------- |--------------|-------------- |  
| 2104            | 5                   | 1                | 45           | 460           |  
| 1416            | 3                   | 2                | 40           | 232           |  
| 852             | 2                   | 1                | 35           | 178           |  

คุณจะสร้างแบบจำลองการถดถอยเชิงเส้นโดยใช้ค่าเหล่านี้เพื่อที่คุณจะสามารถทำนายราคาสำหรับบ้านอื่น ๆ ได้ ตัวอย่างเช่น บ้านขนาด 1200 ตารางฟุต มี 3 ห้องนอน 1 ชั้น อายุ 40 ปี

โปรดเรียกใช้เซลล์โค้ดต่อไปนี้เพื่อสร้างตัวแปร `X_train` และ `y_train` ของคุณ

In [ ]:
X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])

<a name="toc_15456_2.1"></a>
## 2.1 เมทริกซ์ X ที่มีตัวอย่างของเรา
คล้ายกับตารางด้านบน ตัวอย่างจะถูกเก็บในเมทริกซ์ NumPy `X_train` แต่ละแถวของเมทริกซ์แทนตัวอย่างหนึ่งตัว เมื่อคุณมีตัวอย่างการฝึก $m$ ตัว ($m$ คือสามในตัวอย่างของเรา) และมีคุณลักษณะ $n$ ตัว (สี่ในตัวอย่างของเรา) $\mathbf{X}$  คือเมทริกซ์ที่มีมิติ ($m$, $n$) (m แถว n คอลัมน์)

$$\mathbf{X} = 
\begin{pmatrix}
 x^{(0)}_0 & x^{(0)}_1 & \cdots & x^{(0)}_{n-1} \\ 
 x^{(1)}_0 & x^{(1)}_1 & \cdots & x^{(1)}_{n-1} \\
 \cdots \\
 x^{(m-1)}_0 & x^{(m-1)}_1 & \cdots & x^{(m-1)}_{n-1} 
\end{pmatrix}
$$
สัญลักษณ์:
- $\mathbf{x}^{(i)}$ คือเวกเตอร์ที่มีตัวอย่างที่ i. $\mathbf{x}^{(i)}$ $ = (x^{(i)}_0, x^{(i)}_1, \cdots,x^{(i)}_{n-1})$
- $x^{(i)}_j$ คือองค์ประกอบที่ j ในตัวอย่างที่ i อักษรย่อในวงเล็บบ่งชี้หมายเลขตัวอย่างในขณะที่ตัวห้อยแสดงถึงองค์ประกอบ  

แสดงข้อมูลอินพุต



In [ ]:
# data is stored in numpy array/matrix
print(f"X Shape: {X_train.shape}, X Type:{type(X_train)})")
print(X_train)
print(f"y Shape: {y_train.shape}, y Type:{type(y_train)})")
print(y_train)

<a name="toc_15456_2.2"></a>

## 2.2 เวกเตอร์พารามิเตอร์ w, b

* $\mathbf{w}$ เป็นเวกเตอร์ที่มี $n$ องค์ประกอบ
- แต่ละองค์ประกอบมีพารามิเตอร์ที่เกี่ยวข้องกับหนึ่งคุณลักษณะ
- ในชุดข้อมูลของเรา n คือ 4
- โดยนัย เราวาดสิ่งนี้เป็นเวกเตอร์คอลัมน์
$$\mathbf{w} = \begin{pmatrix}
w_0 \\ 
w_1 \\
\cdots\\
w_{n-1}
\end{pmatrix}
$$
* $b$ เป็นพารามิเตอร์สเกลาร์  

เพื่อการสาธิต $\mathbf{w}$ และ $b$ จะถูกโหลดด้วยค่าที่เลือกเริ่มต้นบางค่าที่ใกล้เคียงกับค่าที่เหมาะสมที่สุด $\mathbf{w}$ เป็นเวกเตอร์ NumPy 1 มิติ


In [ ]:
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])
print(f"w_init shape: {w_init.shape}, b_init type: {type(b_init)}")

<a name="toc_15456_3"></a>
# 3 3 การทำนายแบบจำลองด้วยตัวแปรหลายตัว
การทำนายของแบบจำลองด้วยตัวแปรหลายตัวจะถูกกำหนดโดยแบบจำลองเชิงเส้น:

$$ f_{\mathbf{w},b}(\mathbf{x}) =  w_0x_0 + w_1x_1 +... + w_{n-1}x_{n-1} + b \tag{1}$$
หรือในสัญกรณ์เวกเตอร์:
$$ f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b  \tag{2} $$ 
โดยที่  $\cdot$  คือ `ผลคูณดอท` ของเวกเตอร์

เพื่อแสดงผลคูณดอท เราจะนำการทำนายไปใช้งานโดยใช้ (1) และ (2)


<a name="toc_15456_3.1"></a>
## 3.1 การทำนายเดี่ยว องค์ประกอบตามองค์ประกอบ
การทำนายก่อนหน้านี้ของเราคูณค่าคุณลักษณะหนึ่งด้วยพารามิเตอร์หนึ่งและบวกพารามิเตอร์อคติ การขยายโดยตรงของการนำไปใช้ก่อนหน้านี้ของการทำนายไปยังคุณลักษณะหลายตัวจะเป็นการนำไปใช้ (1) ด้านบนโดยใช้ลูปเหนือแต่ละองค์ประกอบ ทำการคูณด้วยพารามิเตอร์ของมันแล้วบวกพารามิเตอร์อคติในตอนท้าย


In [ ]:
def predict_single_loop(x, w, b): 
    """
    single predict using linear regression
    
    Args:
      x (ndarray): Shape (n,) example with multiple features
      w (ndarray): Shape (n,) model parameters    
      b (scalar):  model parameter     
      
    Returns:
      p (scalar):  prediction
    """
    n = x.shape[0]
    p = 0
    for i in range(n):
        p_i = x[i] * w[i]  
        p = p + p_i         
    p = p + b                
    return p

In [ ]:
# get a row from our training data
x_vec = X_train[0,:]
print(f"x_vec shape {x_vec.shape}, x_vec value: {x_vec}")

# make a prediction
f_wb = predict_single_loop(x_vec, w_init, b_init)
print(f"f_wb shape {f_wb.shape}, prediction: {f_wb}")

หมายเหตุรูปร่าง(shape)ของ `x_vec`

`x_vec` เป็นเวกเตอร์ NumPy 1 มิติที่มี 4 องค์ประกอบ (4,) ผลลัพธ์ `f_wb` เป็นสเกลาร์









<a name="toc_15456_3.2"></a>
## 3.2 การทำนายเดี่ยว, เวกเตอร์

เนื่องจากสมการ (1) ด้านบนสามารถนำไปใช้โดยใช้ผลคูณจุดดังเช่น (2) ด้านบน เราสามารถใช้ประโยชน์จากการดำเนินการของเวกเตอร์เพื่อเร่งการทำนาย

จำไว้จากแลป Python/Numpy ว่า `np.dot()`[[link](https://numpy.org/doc/stable/reference/generated/numpy.dot.html)] สามารถใช้เพื่อคำนวณผลคูณจุดของเวกเตอร์


In [ ]:
def predict(x, w, b): 
    """
    single predict using linear regression
    Args:
      x (ndarray): Shape (n,) example with multiple features
      w (ndarray): Shape (n,) model parameters   
      b (scalar):             model parameter 
      
    Returns:
      p (scalar):  prediction
    """
    p = np.dot(x, w) + b     
    return p    

In [ ]:
# get a row from our training data
x_vec = X_train[0,:]
print(f"x_vec shape {x_vec.shape}, x_vec value: {x_vec}")

# make a prediction
f_wb = predict(x_vec,w_init, b_init)
print(f"f_wb shape {f_wb.shape}, prediction: {f_wb}")

ผลลัพธ์และรูปร่างจะเหมือนกับเวอร์ชันก่อนหน้าที่ใช้การวนซ้ำ ต่อไปนี้ `np.dot` จะถูกใช้สำหรับการดำเนินการเหล่านี้ การทำนายตอนนี้เป็นคำสั่งเดียว ฟังก์ชันส่วนใหญ่จะนำไปใช้โดยตรงแทนที่จะเรียกฟังก์ชันการทำนายแยกต่างหาก

<a name="toc_15456_4"></a>
# 4 คำนวณ cost ด้วยตัวแปรหลายตัว
สมการสำหรับฟังก์ชันต้นทุนที่มีตัวแปรหลายตัว $J(\mathbf{w},b)$ คือ:
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{3}$$ 
โดยที่:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{4} $$ 


ซึ่งแตกต่างจากห้องปฏิบัติการก่อนหน้านี้, $\mathbf{w}$ และ  $\mathbf{x}^{(i)}$ ป็นเวกเตอร์แทนที่จะเป็นสเกลาร์ รองรับคุณลักษณะหลายตัว

ด้านล่างนี้เป็นการใช้งานสมการ (3) และ (4) โปรดสังเกตว่านี่ใช้ *รูปแบบมาตรฐานสำหรับหลักสูตรนี้* ซึ่งใช้ลูป for สำหรับตัวอย่างทั้งหมด `m`

In [ ]:
def compute_cost(X, y, w, b): 
    """
    compute cost
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """
    m = X.shape[0]
    cost = 0.0
    for i in range(m):                                
        f_wb_i = np.dot(X[i], w) + b           #(n,)(n,) = scalar (see np.dot)
        cost = cost + (f_wb_i - y[i])**2       #scalar
    cost = cost / (2 * m)                      #scalar    
    return cost

In [ ]:
# Compute and display cost using our pre-chosen optimal parameters. 
cost = compute_cost(X_train, y_train, w_init, b_init)
print(f'Cost at optimal w : {cost}')

**ผลลัพธ์ที่คาดหวัง**: ค่าใช้จ่ายที่ w ที่เหมาะสม: 1.5578904045996674e-12

<a name="toc_15456_5"></a>
# 5 การไล่ระดับลงด้วยตัวแปรหลายตัว
การไล่ระดับลงสำหรับตัวแปรหลายตัว:

$$\begin{align*} \text{ทำซ้ำ}&\text{ จนกระทั่งลู่เข้า:} \; \lbrace \newline\;
& w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{5}  \; & \text{for j = 0..n-1}\newline
&b\ \ = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

โดยที่, n  คือจำนวนคุณลักษณะ พารามิเตอร์ $w_j$,  $b$, จะได้รับการอัปเดตพร้อมกัน และโดยที่  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{6}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{7}
\end{align}
$$
* m คือจำนวนตัวอย่างการฝึกในชุดข้อมูล

    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ คือการทำนายของแบบจำลอง ในขณะที่ $y^{(i)}$ คือค่าเป้าหมาย


<a name="toc_15456_5.1"></a>
## 5.1 คำนวณการไล่ระดับด้วยตัวแปรหลายตัว
การใช้งานสำหรับการคำนวณสมการ (6) และ (7) อยู่ด้านล่าง มีหลายวิธีในการใช้งานนี้ ในเวอร์ชันนี้ มี

- ลูปภายนอกสำหรับตัวอย่างทั้งหมด m ตัว. 
    - $\frac{\partial J(\mathbf{w},b)}{\partial b}$ สำหรับตัวอย่างสามารถคำนวณได้โดยตรงและสะสม
    - ในลูปที่สองสำหรับคุณลักษณะทั้งหมด n ตัว:
        - $\frac{\partial J(\mathbf{w},b)}{\partial w_j}$ คำนวณสำหรับแต่ละ $w_j$.
   

In [ ]:
def compute_gradient(X, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_db, dj_dw

In [ ]:
#Compute and display gradient 
tmp_dj_db, tmp_dj_dw = compute_gradient(X_train, y_train, w_init, b_init)
print(f'dj_db at initial w,b: {tmp_dj_db}')
print(f'dj_dw at initial w,b: \n {tmp_dj_dw}')

**ผลลัพธ์ที่คาดหวัง:**:   
dj_db at เริ่มต้น w,b: -1.6739251122999121e-06  
dj_dw at เริ่มต้น w,b:   
 [-2.73e-03 -6.27e-06 -2.22e-06 -6.92e-05]  

<a name="toc_15456_5.2"></a>
## 5.2 การไล่ระดับลงด้วยตัวแปรหลายตัว
ขั้นตอนด้านล่างนี้ใช้สมการ (5) ด้านบน

In [ ]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    """
    Performs batch gradient descent to learn w and b. Updates w and b by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X (ndarray (m,n))   : Data, m examples with n features
      y (ndarray (m,))    : target values
      w_in (ndarray (n,)) : initial model parameters  
      b_in (scalar)       : initial model parameter
      cost_function       : function to compute cost
      gradient_function   : function to compute the gradient
      alpha (float)       : Learning rate
      num_iters (int)     : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,)) : Updated values of parameters 
      b (scalar)       : Updated value of parameter 
      """
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db,dj_dw = gradient_function(X, y, w, b)   ##None

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               ##None
        b = b - alpha * dj_db               ##None
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( cost_function(X, y, w, b))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")
        
    return w, b, J_history #return final w,b and J history for graphing

ในเซลล์ถัดไป คุณจะทดสอบการใช้งาน

In [ ]:
# initialize parameters
initial_w = np.zeros_like(w_init)
initial_b = 0.
# some gradient descent settings
iterations = 1000
alpha = 5.0e-7
# run gradient descent 
w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b,
                                                    compute_cost, compute_gradient, 
                                                    alpha, iterations)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")
m,_ = X_train.shape
for i in range(m):
    print(f"prediction: {np.dot(X_train[i], w_final) + b_final:0.2f}, target value: {y_train[i]}")

**ผลลัพธ์ที่คาดหวัง:**:    

- b,w พบโดยการไล่ระดับลง: -0.00,[ 0.2   0.   -0.01 -0.07]
- การทำนาย: 426.19, ค่าเป้าหมาย: 460
- การทำนาย: 286.17, ค่าเป้าหมาย: 232
- การทำนาย: 171.47, ค่าเป้าหมาย: 178









In [ ]:
# plot cost versus iteration  
fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, figsize=(12, 4))
ax1.plot(J_hist)
ax2.plot(100 + np.arange(len(J_hist[100:])), J_hist[100:])
ax1.set_title("Cost vs. iteration");  ax2.set_title("Cost vs. iteration (tail)")
ax1.set_ylabel('Cost')             ;  ax2.set_ylabel('Cost') 
ax1.set_xlabel('iteration step')   ;  ax2.set_xlabel('iteration step') 
plt.show()

ผลลัพธ์เหล่านี้ไม่น่าตื่นเต้นเลย! 

cost คงลดลงและการทำนายของเราไม่ค่อยแม่นยำ ห้องปฏิบัติการต่อไปจะสำรวจวิธีการปรับปรุงเรื่องนี้


<a name="toc_15456_6"></a>
# 6 ยินดีด้วย!
ในห้องปฏิบัติการนี้ คุณได้:
- พัฒนาขั้นตอนสำหรับการถดถอยเชิงเส้นอีกครั้ง โดยตอนนี้รองรับหลายตัวแปร
- ใช้งานฟังก์ชัน `np.dot` ของ NumPy เพื่อเวคเตอร์ไรซ์การใช้งาน






